In [0]:
# Load your data
oct_events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

In [0]:
# Distinct products
products_oct = oct_events.select(
    "product_id", "category_id", "category_code", "brand"
).dropDuplicates(["product_id"])

In [0]:
# Distinct users
users_oct = oct_events.select("user_id").dropDuplicates(["user_id"])

In [0]:
# Inner Join
events_with_products_inner = (
    oct_events.drop("brand", "category_code", "price")
    .join(products_oct, on="product_id", how="inner")
)

events_with_products_inner.select(
    "event_time", "event_type", "product_id",
    "brand", "category_code"
).show(5, truncate=False)


+-------------------+----------+----------+------+-----------------------+
|event_time         |event_type|product_id|brand |category_code          |
+-------------------+----------+----------+------+-----------------------+
|2019-10-01 02:21:26|view      |1005159   |xiaomi|electronics.smartphone |
|2019-10-01 02:21:54|view      |1005159   |xiaomi|electronics.smartphone |
|2019-10-01 02:36:20|view      |1005159   |xiaomi|electronics.smartphone |
|2019-10-01 02:38:37|view      |5701087   |jvc   |auto.accessories.player|
|2019-10-01 02:39:03|view      |5701087   |jvc   |auto.accessories.player|
+-------------------+----------+----------+------+-----------------------+
only showing top 5 rows


In [0]:
events_with_products_inner.printSchema()


root
 |-- product_id: integer (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)



In [0]:
#Left Join
from pyspark.sql import functions as F

events_with_products_left = (
    oct_events.drop("brand")        # keep brand only from products_oct
    .join(products_oct, on="product_id", how="left")
)

events_with_products_left.filter(F.col("brand").isNull()).count()

7635545

In [0]:
# Right Join

events_with_products_right = oct_events.join(
    products_oct,
    on="product_id",
    how="right"
)

In [0]:
events_with_products_right.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)



In [0]:
# Outer Join
events_with_products_outer = oct_events.join(
    products_oct,
    on="product_id",
    how="outer"
)
events_with_products_outer.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)



In [0]:
# Running Event count per user
from pyspark.sql.window import Window

user_time_window = Window.partitionBy("user_id").orderBy("event_time")

oct_events_with_cum = oct_events.withColumn(
    "cumulative_events",
    F.count("*").over(user_time_window)
)

oct_events_with_cum.select(
    "user_id", "event_time", "event_type", "cumulative_events"
).orderBy("user_id", "event_time").show(20, truncate=False)


+---------+-------------------+----------+-----------------+
|user_id  |event_time         |event_type|cumulative_events|
+---------+-------------------+----------+-----------------+
|33869381 |2019-10-23 20:04:08|view      |1                |
|64078358 |2019-10-13 00:13:46|view      |1                |
|183503497|2019-10-02 21:43:00|view      |1                |
|184265397|2019-10-04 17:44:37|view      |1                |
|184265397|2019-10-04 17:45:18|view      |2                |
|184265397|2019-10-04 17:50:50|view      |3                |
|184265397|2019-10-04 17:51:04|view      |4                |
|184265397|2019-10-15 17:18:59|view      |5                |
|184265397|2019-10-15 17:19:28|view      |6                |
|195082191|2019-10-10 03:35:36|view      |1                |
|200673532|2019-10-10 15:02:36|view      |1                |
|200673532|2019-10-12 07:58:37|view      |2                |
|200673532|2019-10-12 14:47:07|view      |3                |
|200673532|2019-10-13 14

In [0]:
# Rank products by revenue per category
# First, revenue per product+category
product_cat_revenue = oct_events.filter(F.col("event_type") == "purchase") \
    .groupBy("category_code", "product_id", "brand") \
    .agg(F.sum("price").alias("revenue"))

# Rank within each category
cat_window = Window.partitionBy("category_code").orderBy(F.desc("revenue"))

ranked_products = product_cat_revenue.withColumn(
    "revenue_rank",
    F.dense_rank().over(cat_window)
)

# Top 5 per category
top5_per_category = ranked_products.filter(F.col("revenue_rank") <= 5)

top5_per_category.orderBy("category_code", "revenue_rank").show(50, truncate=False)


+--------------------+----------+-----------+------------------+------------+
|category_code       |product_id|brand      |revenue           |revenue_rank|
+--------------------+----------+-----------+------------------+------------+
|NULL                |4100346   |sony       |471169.62999999995|1           |
|NULL                |4100129   |sony       |230103.49000000002|2           |
|NULL                |5100816   |xiaomi     |208635.80000000008|3           |
|NULL                |26300094  |lucente    |198204.56         |4           |
|NULL                |26300354  |lucente    |163844.5          |5           |
|accessories.bag     |28401058  |karya      |1505.8500000000004|1           |
|accessories.bag     |21000006  |thule      |1194.28           |2           |
|accessories.bag     |28400775  |baden      |861.2799999999999 |3           |
|accessories.bag     |28401252  |NULL       |686.26            |4           |
|accessories.bag     |18300846  |roncato    |615.2             |

In [0]:
# Weekend flag and hour of the day     
oct_events_features = oct_events \
    .withColumn("event_date", F.to_date("event_time")) \
    .withColumn("event_hour", F.hour("event_time")) \
    .withColumn("day_of_week", F.date_format("event_time", "E")) \
    .withColumn(
        "is_weekend",
        F.when(F.col("day_of_week").isin("Sat", "Sun"), 1).otherwise(0)
    )

oct_events_features.select(
    "event_time", "day_of_week", "is_weekend", "event_hour"
).show(10, truncate=False)


+-------------------+-----------+----------+----------+
|event_time         |day_of_week|is_weekend|event_hour|
+-------------------+-----------+----------+----------+
|2019-10-01 00:00:00|Tue        |0         |0         |
|2019-10-01 00:00:00|Tue        |0         |0         |
|2019-10-01 00:00:01|Tue        |0         |0         |
|2019-10-01 00:00:01|Tue        |0         |0         |
|2019-10-01 00:00:04|Tue        |0         |0         |
|2019-10-01 00:00:05|Tue        |0         |0         |
|2019-10-01 00:00:08|Tue        |0         |0         |
|2019-10-01 00:00:08|Tue        |0         |0         |
|2019-10-01 00:00:10|Tue        |0         |0         |
|2019-10-01 00:00:11|Tue        |0         |0         |
+-------------------+-----------+----------+----------+
only showing top 10 rows


In [0]:
# Price Buckets
oct_events_features = oct_events_features.withColumn(
    "price_bucket",
    F.when(F.col("price") < 50, "low")
     .when((F.col("price") >= 50) & (F.col("price") < 200), "medium")
     .otherwise("high")
)

oct_events_features.groupBy("price_bucket").count().show()


+------------+--------+
|price_bucket|   count|
+------------+--------+
|         low| 7974186|
|      medium|16416741|
|        high|18057837|
+------------+--------+



In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Top 5 products by revenue
revenue = oct_events.filter(F.col("event_type") == "purchase") \
    .groupBy("product_id", "product_name") \
    .agg(F.sum("price").alias("revenue")) \
    .orderBy(F.desc("revenue")).limit(5)

# Running total per user
window = Window.partitionBy("user_id").orderBy("event_time")
oct_events.withColumn("cumulative_events", F.count("*").over(window))

conv_df = (
    oct_events
        .groupBy("category_code", "event_type")
        .count()
        .groupBy("category_code")
        .pivot("event_type")
        .sum("count")
        .withColumn("conversion_rate", F.col("purchase") / F.col("view") * 100)
)

conv_df.show()

+--------------------+-----+--------+-------+-------------------+
|       category_code| cart|purchase|   view|    conversion_rate|
+--------------------+-----+--------+-------+-------------------+
|auto.accessories....| NULL|      46|  12305| 0.3738317757009346|
|furniture.living_...| NULL|    1084| 215471| 0.5030839416905292|
| stationery.cartrige|  106|     134|   7380| 1.8157181571815717|
|       sport.bicycle|  693|     838| 128759| 0.6508282916145668|
|        apparel.sock|    7|      21|   2621| 0.8012209080503624|
|appliances.enviro...|   16|      27|   2172| 1.2430939226519337|
|          kids.swing|  147|     330|  31596|  1.044436004557539|
|electronics.audio...|  196|     430|  28394| 1.5144044516447137|
|auto.accessories....|  716|     494|  42350|  1.166469893742621|
|  electronics.clocks|20344|   17906|1272783| 1.4068384005757462|
|electronics.audio...|  410|     423|  35409| 1.1946115394391257|
|appliances.kitche...| 1080|     936| 105149| 0.8901653843593377|
|appliance